# Data Engineering Capstone Project

#### Project Summary
The Capstone project analyses and process data from Brazilian stock market since 2013 until now. Also,
the Brazilian economic data from [World Bank](https://data.worldbank.org/).

With those data in mind, it is possible to evaluate and take insights, relate country data with stock market.
Basically to answer questions like:
* Does the stock market influence real economy?
* Is the population's education rate related to the increase in GDP?
* The raising of company in stock market help us somehow?


The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
!pip install bovespa
!pip install pyspark
!pip install psycopg2
!pip install wbgapi
!pip install matplotlib
!pip install configparser
!pip install pyarrow

^C
Traceback (most recent call last):
  File "/home/charl3ff/workspace/udacity/capstone-udacity-data-engineer/env/bin/pip", line 8, in <module>
    sys.exit(main())
  File "/home/charl3ff/workspace/udacity/capstone-udacity-data-engineer/env/lib/python3.8/site-packages/pip/_internal/cli/main.py", line 73, in main
    return command.main(cmd_args)
  File "/home/charl3ff/workspace/udacity/capstone-udacity-data-engineer/env/lib/python3.8/site-packages/pip/_internal/cli/base_command.py", line 111, in main
    return self._main(args)
  File "/home/charl3ff/workspace/udacity/capstone-udacity-data-engineer/env/lib/python3.8/site-packages/pip/_internal/cli/base_command.py", line 226, in _main
    self.handle_pip_version_check(options)
  File "/home/charl3ff/workspace/udacity/capstone-udacity-data-engineer/env/lib/python3.8/site-packages/pip/_internal/cli/req_command.py", line 151, in handle_pip_version_check
    pip_self_version_check(session, options)
  File "/home/charl3ff/workspace/udacity/c

UnboundLocalError: local variable 'child' referenced before assignment

In [ ]:
# Do all imports and installs here
import pandas as pd
import bovespa
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
import psycopg2
import psycopg2.extras
import wbgapi as wb
import configparser
from pyspark.sql.functions import col, collect_set, sum, avg, max, countDistinct, count

### Step 1: Scope the Project and Gather Data

#### Scope
The Capstone project analyses and process data from Brazilian stock market since 2013 until now. Also,
the Brazilian economic data from [World Bank](https://data.worldbank.org/).

#### Describe and Gather Data
Describe the data sets you're using. Where did it come from? What type of information is included?

Bovespa is the main Brazilian stock Exchange. Here is listed all trading data since 2013.

It includes the day of the trading, company name, stock id, stock data, such as: price open, price close, price high, price low.

World Bank collects all data from all country in the world, since 1960.
It has information about environment, education, GDP, economy, social.
http://www.b3.com.br/en_us/market-data-and-indices/data-services/market-data/historical-data/equities/historical-quotes/
https://api.worldbank.org/v2/en/country/BRA?downloadformat=csv


In [ ]:
config = configparser.ConfigParser()
config.read('config.cfg')

os.environ['AWS_ACCESS_KEY_ID'] = config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = config['AWS']['AWS_SECRET_ACCESS_KEY']

In [ ]:
os.environ['JAVA_HOME'] = ''
spark = SparkSession.builder.enableHiveSupport().config('spark.sql.execution.arrow.pyspark.enabled',
                                                        "true").getOrCreate()

### ETL stock market

In [ ]:
# Load data trading data from bovespa

input_data = "s3://capstone-data-1/"
trading_file = input_data + "COTAHIST_*.txt"

df = spark.read.text(trading_file)
df_pd = df.toPandas()

In [ ]:
# Parse data

def record_to_dict(record):
    try:
        record = bovespa.Record(record)
    except:
        return None
    return {
        'date': record.date, 'year': record.date.year, 'month': record.date.month, 'day': record.date.day,
        'money_volume': record.volume, 'volume': record.quantity,
        'stock_code': record.stock_code,
        'company_name': record.company_name, 'price_open': record.price_open,
        'price_close': record.price_close, 'price_mean': record.price_mean,
        'price_high': record.price_high, 'price_low': record.price_low,
        'variation': None
    }


In [ ]:
# Load trading data into our schema

df_pd['dict'] = df_pd['value'].apply(lambda x: record_to_dict(x))
new_df = df_pd['dict'].apply(pd.Series).replace('', None)

schema = StructType([
    StructField('date', DateType(), True),
    StructField('year', IntegerType(), True),
    StructField('month', IntegerType(), True),
    StructField('day', IntegerType(), True),
    StructField('money_volume', FloatType(), True),
    StructField('volume', IntegerType(), True),
    StructField('stock_code', StringType(), True),
    StructField('company_name', StringType(), True),
    StructField('price_open', FloatType(), True),
    StructField('price_close', FloatType(), True),
    StructField('price_mean', FloatType(), True),
    StructField('price_high', FloatType(), True),
    StructField('price_low', FloatType(), True),
    StructField('variation', FloatType(), True)
])

trading_df = spark.createDataFrame(new_df, schema=schema)

In [ ]:
# Set variation column

trading_df = trading_df.withColumn('variation', (col('price_close') - col('price_open')) / 100)

trading_df.show(10)

### So, we can take all insights from it.
Such as:

In [ ]:
# Most traded stocks in terms of money
trading_df.orderBy('money_volume', ascending=False).show(20)

In [ ]:
# Most up variation in a day
trading_df.orderBy('variation', ascending=False).show(20)

### ETL World Bank

The `wbgapi` package is the official Python/R library to interact with the [World Bank open data](https://data.worldbank.org/),
it is useful to easily retrieve data, instead of download it ourselves.

The purpose is to fetch data from Brazil, then it will be possible to compare the stock market and the real economy.

In [ ]:
# Metrics related to gdp
wb.series.info(q='gdp')

# Metrics related to debt, and so on
wb.series.info(q='debt')

In [ ]:
metric_to_schema = {
    'GC.DOD.TOTL.GD.ZS': 'debt',
    'GC.XPN.TOTL.CN': 'total_expense',
    'NY.GDP.MKTP.KD.ZG': 'gdp_annual_growth',
    'NY.GDP.MKTP.CD': 'gdp',
    'NY.GDP.PCAP.CD': 'gdp_per_capita',
    'SP.POP.TOTL': 'population',
    'SP.DYN.LE00.IN': 'life_expectancy',
    'GC.XPN.TOTL.GD.ZS': 'expense_per_gdp',
    'FI.RES.TOTL.CD': 'total_reserves',
    'SE.ADT.LITR.ZS': 'pop_literacy_rate',
    'SE.XPD.TOTL.GD.ZS': 'expenditure_education_per_gdp',
    'BX.KLT.DINV.CD.WD': 'foreign_investment',
}

In [2]:
# Load data into pandas
country = 'BRA'
values = []
for key, value in metric_to_schema.items():
    df = wb.data.DataFrame(key, economy=country, columns='time', numericTimeKeys=True).dropna(axis=1, how='all')
    for data in df.items():
        values.append((data[0], value, data[1][country]))
economy_df = pd.DataFrame(data=values, columns=['year', 'table', 'value'])
economy_df = spark.createDataFrame(economy_df)

NameError: name 'metric_to_schema' is not defined

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [ ]:
# Performing cleaning tasks here




### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

### Trading Table

| Feature        | Description  |
| ------------- |:-------------:|
| date         | date that stock is being traded                      |
| year         | year that stock is being traded                      |
| month        | month that stock is being traded                     |
| day          | day that stock is being traded                       |
| money_volume | total amount of money in all transactions            |
| volume       | total number traded stocks                           |
| stock_code   | unique stock code                                    |
| company_name | name of the company that is represented by the stock |
| price_open   | price when trading opened                            |
| price_close  | price when trading closed                            |
| price_mean   | price mean of the stock                              |
| price_high   | price high of the stock                              |
| price_low    | price low of the stock                               |
| variation    | the variation between price close and price open     |

### Economy Table

| Feature        | Description  |
| ------------- |:-------------:|
| year  | the year the statics was collected           |
| table | the statics itself, like, GDP, debt over GDP |
| value | the value of the statics                     |

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.